email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [ ]:
response = agent.invoke(
            {"messages": [HumanMessage(content="Please check my inbox.")]},
            context = EmailContext(),
            config = config
)

print(response['messages'][-1].content)

In [17]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
            {"messages": [HumanMessage(content="julie@example.com, password123")]},
            context = EmailContext(),
            config = config
)

print(response['messages'][-1].content)

/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


I found 1 new email from Jane (jane@example.com): “Hi Julie, I’m going to be in town next week and was wondering if we could grab a coffee?”

Would you like me to draft and send a reply? I can tailor a few quick options. Here are some templates you can choose from or we can customize:

Option 1 — Casual and upbeat
Hi Jane,
Coffee sounds great! I’m around next week. How about [time] on [day] at [cafe]? If that doesn’t work, tell me what times you have in mind.
Cheers,
Julie

Option 2 — Short and flexible
Hi Jane, I’d love to. I’m available [times]. Which works for you?

Option 3 — If you want a specific location
Hi Jane, I’d love to catch up. How about [cafe/area] next week? I’m free [days/times]. Let me know what works.

Tips:
- Replace [day/time/cafe] with your actual availability and a nearby cafe.
- If you prefer a firm plan, propose a couple of exact slots.

Would you like me to send one of these now? If you share your availability and preferred cafe, I’ll draft a final version and

In [18]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="draft 1")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

Draft 1 – Reply to Jane’s coffee invitation (casual and upbeat)

Subject: Re: Coffee next week

Hi Jane,

Coffee sounds great! I’m around next week. How about [time] on [day] at [cafe]? If that doesn’t work, tell me what times you have in mind.

Cheers,
Julie

Notes:
- Replace [time], [day], and [cafe] with your actual availability.
- If you’d prefer a firmer plan, propose a couple of exact slots.

Would you like me to customize this with your preferred times and a specific cafe, and send it? I can also add a calendar invite after you confirm.


In [19]:
#print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

In [20]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "draft 1"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

Draft 1 – Reply to Jane’s coffee invitation (casual and upbeat)

Subject: Re: Coffee next week

Hi Jane,

Coffee sounds great! I’m around next week. How about [time] on [day] at [cafe]? If that doesn’t work, tell me what times you have in mind.

Cheers,
Julie

Notes:
- Replace [time], [day], and [cafe] with your actual availability.
- If you’d prefer a firmer plan, propose a couple of exact slots.

Would you like me to customize this with your preferred times and a specific cafe, and send it? I can also add a calendar invite after you confirm.


In [24]:
from langgraph.types import Command

response = agent.invoke(
    {"messages": [HumanMessage(content="Use 10am Tuesday at Cannelle (or suggest a cafe). Update the draft and show me the final email only.")]},
    config=config
)

print(response["messages"][-1].content)

Subject: Re: Coffee next week

Hi Jane,

Coffee sounds great! I’m around next week. How about 10:00 AM on Tuesday at Cannelle? If that doesn’t work, tell me what times you have in mind.

Cheers,
Julie


In [25]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

Subject: Re: Coffee next week

Hi Jane,

Coffee sounds great! I’m around next week. How about 10:00 AM on Tuesday at Cannelle? If that doesn’t work, tell me what times you have in mind.

Cheers,
Julie


In [26]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='draft 1', additional_kwargs={}, response_metadata={}, id='e0680448-4bac-4dd3-9cda-325ac333b3df'),
              AIMessage(content='I’m happy to help with Draft 1. Could you share a bit more about what you’re drafting? Specifically:\n- Type: email, letter, report, proposal, blog post, etc.\n- Topic or purpose\n- Audience\n- Tone (formal, friendly, persuasive, etc.)\n- Length or word count\n- Any key points or must-have details\n- Deadline or anything else I should know\n\nIf you’d like, I can also provide a ready-to-edit generic Draft 1 now. Here’s a simple, adaptable template you can customize:\n\nDraft 1 – Professional update (email or memo)\nSubject: [Project/Topic] – Draft 1\n\nHi [Name/Team],\n\nI’m sharing an initial draft of [Project/Topic] to gather your input. Here’s where things stand:\n\n- Objective: [Brief objective]\n- Current status: [What’s done, what’s in progress]\n- Key milestones completed: [List]\n- Upcoming